In [1]:
import math

# ====== PARÁMETROS DEL ENUNCIADO ======
N_ROOTS = 5      # "realice 5 raíces cuadradas"
DECIMALS = 6     # "redondeando a 6 cifras"

ALPHA = 1 / math.log(10)  # 1/ln(10)


def briggs_log10(x, n_roots=N_ROOTS, decimals=DECIMALS, verbose=False):
    """
    Aproxima log10(x) con Briggs:
      1) y = x^(1/2^n) por n raíces cuadradas consecutivas
      2) redondea en cada raíz a 'decimals'
      3) log10(y) ≈ ALPHA*(y-1)
      4) log10(x) = 2^n * log10(y)
    """
    y = float(x)
    if verbose:
        print(f"x = {x}")
    for k in range(1, n_roots + 1):
        y = math.sqrt(y)
        y = round(y, decimals)
        if verbose:
            print(f"  raíz {k}: y = {y}")

    r = y - 1.0
    log10_y_aprox = ALPHA * r
    log10_x_aprox = (2 ** n_roots) * log10_y_aprox
    return log10_x_aprox, y, r


def rel_error(aprox, real):
    if real == 0:
        return 0.0
    return abs(aprox - real) / abs(real)


# ====== 1) Calcula con Briggs para 2, 3, 7 ======
log2, y2, r2 = briggs_log10(2)
log3, y3, r3 = briggs_log10(3)
log7, y7, r7 = briggs_log10(7)

# ====== 2) Usa propiedades para el resto (según tu tabla/pistas) ======
aprox = {}
aprox[1] = 0.0
aprox[10] = 1.0

aprox[2] = log2
aprox[3] = log3
aprox[7] = log7

aprox[4] = 2 * log2           # 4 = 2^2
aprox[5] = 1 - log2           # 5 = 10/2
aprox[6] = log2 + log3        # 6 = 2*3
aprox[8] = 3 * log2           # 8 = 2^3
aprox[9] = 2 * log3           # 9 = 3^2

# ====== 3) Imprime tabla con error relativo contra Python ======
print("x | log10_aprox      | log10_real       | error_relativo")
print("--|------------------|-----------------|----------------")
for x in range(1, 11):
    real = math.log10(x)
    err = rel_error(aprox[x], real)
    print(f"{x:>2} | {aprox[x]:>16.12f} | {real:>15.12f} | {err:.6f}")

# ====== 4) Máximo error absoluto de la aproximación log10(1+r) ≈ ALPHA*r ======
# "tolerancia" natural aquí: el peor r que aparece si haces 5 raíces hasta llegar cerca de 1.
# Para enteros 1..10, el mayor r sale con x=10 (porque 10^(1/32) es el mayor).
r_max = (10 ** (1 / (2 ** N_ROOTS))) - 1

# error(r) = |log10(1+r) - ALPHA*r|
max_abs_error = abs(math.log10(1 + r_max) - ALPHA * r_max)

print("\nAproximación lineal log10(1+r) ≈ ALPHA*r")
print(f"r_max (para x=10 con {N_ROOTS} raíces) = {r_max:.12f}")
print(f"Máximo error absoluto en [0, r_max] = {max_abs_error:.12f}")

# ====== 5) (Extra) Comparar dos aproximaciones para log10(6) ======
# A) por propiedades: log6 = log2 + log3
aprox6_prop = aprox[6]

# B) usando la pista 6^9 = 1.0077696 × 10^7
# log10(6) = (log10(1.0077696) + 7) / 9, y log10(1.0077696) ≈ ALPHA*0.0077696
r = 0.0077696
aprox_log_10077696 = ALPHA * r
aprox6_pow = (7 + aprox_log_10077696) / 9

real6 = math.log10(6)
print("\nComparación para log10(6):")
print(f"Real                = {real6:.12f}")
print(f"A) log2+log3        = {aprox6_prop:.12f}  (err={rel_error(aprox6_prop, real6):.6e})")
print(f"B) usando 6^9 pista  = {aprox6_pow:.12f}  (err={rel_error(aprox6_pow, real6):.6e})")


x | log10_aprox      | log10_real       | error_relativo
--|------------------|-----------------|----------------
 1 |   0.000000000000 |  0.000000000000 | 0.000000
 2 |   0.304311880648 |  0.301029995664 | 0.010902
 3 |   0.485409205245 |  0.477121254720 | 0.017371
 4 |   0.608623761295 |  0.602059991328 | 0.010902
 5 |   0.695688119352 |  0.698970004336 | 0.004695
 6 |   0.789721085893 |  0.778151250384 | 0.014868
 7 |   0.871326756220 |  0.845098040014 | 0.031036
 8 |   0.912935641943 |  0.903089986992 | 0.010902
 9 |   0.970818410491 |  0.954242509439 | 0.017371
10 |   1.000000000000 |  1.000000000000 | 0.000000

Aproximación lineal log10(1+r) ≈ ALPHA*r
r_max (para x=10 con 5 raíces) = 0.074607828321
Máximo error absoluto en [0, r_max] = 0.001151768147

Comparación para log10(6):
Real                = 0.778151250384
A) log2+log3        = 0.789721085893  (err=1.486836e-02)
B) usando 6^9 pista  = 0.778152699379  (err=1.862099e-06)
